In [1]:
#@title Bibliotecas padrões
import pandas as pd
import pandas.io.sql as sqlio
import sys
import psycopg2
from ydata_profiling import ProfileReport
import numpy as np
from botocore import UNSIGNED
from botocore.config import Config
from urllib.parse import quote_plus
from pyathena import connect
import boto3
import boto3.session
from botocore import exceptions
import seaborn as sns
import plotly.express as px

#Carregando as bibliotecas necessárias para a análise estatística
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (8, 8)
import statsmodels.api as sm
import statsmodels.tsa as tsa
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import pmdarima as pm
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_process import arma_generate_sample

In [2]:
#ponte de conexão com o redshift
connection = psycopg2.connect(dbname="dw_amorsaude",
                  user="",
                  password="",
                  host ="amazonaws.com",
                  port = 5439)

In [3]:
#query
df_agendamentos = '''select ag."data" as data_agendamento, ag.hora, sac.nome_canal, sas.nome_subcanal, 
u.nome_fantasia as nome_unidade, u.cidade as cidade
, u.id as id_unidade, sp.id as id_paciente, regexp_replace(sp.celular, '[^0-9]', '') as celular, 
es.nome_especialidade, count(*) as qtde
from stg_agendamento_procedimentos ap
left join stg_agendamentos ag on ap.agendamento_id = ag.id
left join stg_pacientes sp on sp.id = ag.paciente_id
left join stg_agendamento_status ass on ag.status_id = ass.id
left join stg_especialidades es on ag.especialidade_id = es.id
left join stg_procedimentos pro on ap.procedimento_id = pro.id
left join stg_procedimentos_tipos pt on pro.tipo_procedimento_id = pt.id
left join stg_locais l on ap.local_id = l.id
left join stg_unidades u on l.unidade_id = u.id
left join stg_agendamento_canais sac on sac.id = ag.canal_id
left join stg_agendamento_subcanais sas on sas.id = ag.subcanal_id 
where pt.id in (2, 9)
and ag.status_id in (33, 207, 202, 2, 200, 203, 5, 204, 201, 205, 4, 206, 3)
and ag."data" between  '2023-11-08' and '2023-11-13'
group by ag."data", sac.nome_canal, sas.nome_subcanal, u.nome_fantasia, 
u.cidade, u.id, sp.celular, ag.hora, es.nome_especialidade, sp.id'''

In [19]:
df_agendamentos = pd.read_sql_query(df_agendamentos,con=connection)

/tmp/ipykernel_7787/3190376523.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_agendamentos = pd.read_sql_query(df_agendamentos,con=connection)


In [70]:
df_agendamentos

,data_agendamento,hora,nome_canal,nome_subcanal,nome_unidade,cidade,id_unidade,id_paciente,celular,nome_especialidade,qtde
0,2023-11-09,12:30:00,App Cartão de Todos,None,AmorSaúde Curitiba Centro,Curitiba,19283,67146818,41987044004,Clinica Médica,1
1,2023-11-10,15:00:00,App Cartão de Todos,None,AmorSaúde Piraquara,Piraquara,19910,65774335,41996883699,Clinica Médica,1
2,2023-11-08,11:30:00,Clínica,Presencial - Clínica,AmorSaúde Maceió Norte,Maceió,19878,66066492,82993011436,Ginecologia e Obstetrícia,1
3,2023-11-11,08:45:00,Call Center Nacional,None,AmorSaúde Salvador Largo do Tanque,Salvador,19726,59279174,71987397412,Oftalmologia,1
4,2023-11-08,14:30:00,Clínica,Call Center - Clínica,AmorSaúde Carapicuíba,Carapicuíba,18263,65137934,11937050380,Psicologia,1
...,...,...,...,...,...,...,...,...,...,...,...
269896,2023-11-13,14:15:00,Clínica,Call Center - Clínica,AmorSaúde Macaé,Macaé,19462,67537713,22998565146,Angiologia,1
269897,2023-11-13,14:10:00,App Cartão de Todos,None,AmorSaúde Betim,Betim,19314,67409333,31991267016,Oftalmologia,1
269898,2023-11-08,18:30:00,Clínica,Call Center - Clínica,AmorSaúde Arapiraca,Arapiraca,19798,67602403,82987361602,Ginecologia e Obstetrícia,1
269899,2023-11-13,14:30:00,Clínica,Presencial - Clínica,AmorSaúde RJ Madureira,Rio De Janeiro,19655,56982440,21972723038,Fisioterapia,1


In [40]:
# Substitua 'seu_arquivo.xlsx' pelo caminho do seu arquivo Excel
caminho_arquivo = '/home/gabriel/dbt-projects/python_files/analises/files/analitycs_rcs_final.csv'

# Leia o arquivo Excel
df_crm = pd.read_csv(caminho_arquivo, delimiter=';')
df_crm = df_crm.rename(columns={'ID do Paciente': 'id_paciente'})
df_crm['id_paciente'] = df_crm['id_paciente'].replace('#N/D', pd.NA)
df_crm = df_crm.dropna(subset=['id_paciente'])
df_crm['id_paciente'] = df_crm['id_paciente'].str.replace('.', '', regex=False).astype(int)

In [60]:
#Separando os #N/D
caminho_arquivo = '/home/gabriel/dbt-projects/python_files/analises/files/analitycs_rcs_final.csv'

# Leia o arquivo Excel
df_crm_cel = pd.read_csv(caminho_arquivo, delimiter=';')
df_crm_cel = df_crm_cel.rename(columns={'ID do Paciente': 'id_paciente'})
df_crm_cel = df_crm_cel.loc[df_crm_cel['id_paciente'] == '#N/D']
df_crm_cel = df_crm_cel.rename(columns={'NUMERO': 'celular'})

In [62]:
chunk_merge = pd.merge(df_crm, df_agendamentos, on='id_paciente', how='inner')

In [72]:
df_agendamentos_celular = '''select ag."data" as data_agendamento, ag.hora, sac.nome_canal, sas.nome_subcanal, 
u.nome_fantasia as nome_unidade, u.cidade as cidade
, u.id as id_unidade, sp.id as id_paciente, regexp_replace(sp.celular, '[^0-9]', '') as celular, 
es.nome_especialidade, count(*) as qtde
from stg_agendamento_procedimentos ap
left join stg_agendamentos ag on ap.agendamento_id = ag.id
left join stg_pacientes sp on sp.id = ag.paciente_id
left join stg_agendamento_status ass on ag.status_id = ass.id
left join stg_especialidades es on ag.especialidade_id = es.id
left join stg_procedimentos pro on ap.procedimento_id = pro.id
left join stg_procedimentos_tipos pt on pro.tipo_procedimento_id = pt.id
left join stg_locais l on ap.local_id = l.id
left join stg_unidades u on l.unidade_id = u.id
left join stg_agendamento_canais sac on sac.id = ag.canal_id
left join stg_agendamento_subcanais sas on sas.id = ag.subcanal_id 
where pt.id in (2, 9)
and ag.status_id in (33, 207, 202, 2, 200, 203, 5, 204, 201, 205, 4, 206, 3)
and ag."data" between  '2023-11-08' and '2023-11-13'
and regexp_replace(sp.celular, '[^0-9]', '') in (11997079798,
19995353913,
19998235192,
19992246193,
19981800685,
19996341944,
19992917339,
19992512173,
19987211939,
19998642825,
11937433678,
19989933205,
19981436907,
19992611938,
19992741845,
19992998091,
19999447272,
19981847495,
19995365316,
19996720416,
11985062126,
19993031138,
19987422082,
19993077229,
19996992400,
19988063520,
19988405606,
19996346975,
19995098594,
19982489944,
19984175846,
19981964397,
19995394643,
19994725133,
19994255645,
19981511638,
19974238283,
19987401024,
19981526872,
19998482974,
19981299398,
19986044577,
19992377824,
19995983318,
19998249884,
19999823331,
33999830947)
group by ag."data", sac.nome_canal, sas.nome_subcanal, u.nome_fantasia, 
u.cidade, u.id, sp.celular, ag.hora, es.nome_especialidade, sp.id'''

In [75]:
df_agendamentos_celular = pd.read_sql_query(df_agendamentos_celular,con=connection)

/tmp/ipykernel_7787/413633568.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_agendamentos_celular = pd.read_sql_query(df_agendamentos_celular,con=connection)


In [76]:
chunk_merge

,NUMERO,STATUS,id_paciente,data_agendamento,hora,nome_canal,nome_subcanal,nome_unidade,cidade,id_unidade,celular,nome_especialidade,qtde
0,11963510703,LIDO,67099592,2023-11-10,09:42:00,Clínica,Presencial - Clínica,AmorSaúde SP Grajaú,São Paulo,19821,11963510703,Ortopedia e Traumatologia,1
1,71993246063,LIDO,63655647,2023-11-11,08:02:00,Clínica,Presencial - Clínica,AmorSaúde Salvador Cabula,Salvador,19818,71991758401,Ginecologia,1
2,71983185999,LIDO,60580509,2023-11-11,08:11:00,Clínica,Call Center - Clínica,AmorSaúde Salvador Cabula,Salvador,19818,71983185999,Urologia,1
3,71996236047,LIDO,64420892,2023-11-08,14:15:00,Clínica,Presencial - Clínica,AmorSaúde Salvador Paripe,Salvador,19806,71996236047,Oftalmologia,1
4,71996236047,LIDO,64420892,2023-11-08,14:40:00,App Cartão de Todos,None,AmorSaúde Salvador Paripe,Salvador,19806,71996236047,Gastroenterologia,1
5,71987529281,LIDO,57858911,2023-11-13,14:05:00,Clínica,Presencial - Clínica,AmorSaúde Salvador Centro,Salvador,19270,71987529281,Cardiologia,1
6,11989579939,LIDO,59605171,2023-11-13,14:20:00,Clínica,Presencial - Clínica,AmorSaúde SP Grajaú,São Paulo,19821,11989579939,Dermatologia,1
7,71981925356,LIDO,67495379,2023-11-11,08:56:00,Clínica,Presencial - Clínica,AmorSaúde Salvador Largo do Tanque,Salvador,19726,71981925356,Clinica Médica,1
8,71986398764,LIDO,67462712,2023-11-08,09:50:00,Clínica,Call Center - Clínica,AmorSaúde Salvador Centro,Salvador,19270,71986398764,Cardiologia,1
9,71986398764,LIDO,67462712,2023-11-10,17:00:00,Clínica,Call Center - Clínica,AmorSaúde Salvador Centro,Salvador,19270,71986398764,Psicologia,1


In [77]:
df_agendamentos_celular

,data_agendamento,hora,nome_canal,nome_subcanal,nome_unidade,cidade,id_unidade,id_paciente,celular,nome_especialidade,qtde
0,2023-11-09,10:30:00,Call Center Nacional,None,AmorSaúde Campinas Centro,Campinas,19934,11604655,19981847495,Clinica Médica,1
1,2023-11-08,14:15:00,Clínica,Whatspp - Clínica,AmorSaúde Campinas Centro,Campinas,19934,63805447,19988405606,Psiquiatria,1


In [78]:
nome_do_arquivo = 'python_analitycs_rcs_final.xlsx'
chunk_merge.to_excel(nome_do_arquivo, index=False)  # O argumento 'index=False' evita salvar o índice do DataFrame no Excel

In [79]:
nome_do_arquivo = 'python_analitycs_rcs_celular.xlsx'
df_agendamentos_celular.to_excel(nome_do_arquivo, index=False)  # O argumento 'index=False' evita salvar o índice do DataFrame no Excel